In [44]:
# Import all necessary libraries 

import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 
#import warnings
#warnings.filterwarnings("ignore")

# Data Exploration

In [47]:
# Load data
data = pd.read_csv("./dataset.csv")
data = data.astype(int)
data

,Diabetes_012,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,Veggies,HvyAlcoholConsump,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income
0,0,1,1,1,40,1,0,0,0,0,1,0,1,0,5,18,15,1,0,9,4,3
1,0,0,0,0,25,1,0,0,1,0,0,0,0,1,3,0,0,0,0,7,6,1
2,0,1,1,1,28,0,0,0,0,1,0,0,1,1,5,30,30,1,0,9,4,8
3,0,1,0,1,27,0,0,0,1,1,1,0,1,0,2,0,0,0,0,11,3,6
4,0,1,1,1,24,0,0,0,1,1,1,0,1,0,2,3,0,0,0,11,5,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
253675,0,1,1,1,45,0,0,0,0,1,1,0,1,0,3,0,5,0,1,5,6,7
253676,2,1,1,1,18,0,0,0,0,0,0,0,1,0,4,0,0,1,0,11,2,4
253677,0,0,0,1,28,0,0,0,1,1,0,0,1,0,1,0,0,0,0,2,5,2
253678,0,1,0,1,23,0,0,0,0,1,1,0,1,0,3,0,0,0,1,7,5,1


### Data types exploration

In [ ]:
data_types = data.dtypes
data_types

Diabetes_012            int64
HighBP                  int64
HighChol                int64
CholCheck               int64
BMI                     int64
Smoker                  int64
Stroke                  int64
HeartDiseaseorAttack    int64
PhysActivity            int64
Fruits                  int64
Veggies                 int64
HvyAlcoholConsump       int64
AnyHealthcare           int64
NoDocbcCost             int64
GenHlth                 int64
MentHlth                int64
PhysHlth                int64
DiffWalk                int64
Sex                     int64
Age                     int64
Education               int64
Income                  int64
dtype: object

The 'Diabetes_012' has 3 categories, 
'GenHlth' has 5 cateogries, which are nominal (1 = excellent to 5 = poor),

'MentHlth' ??? -> sorted but categorical ??? 30 possible
'PhysHlth' ??? -> 30 possible

'Age', Education, Income


the rest of the cate_cols variables have 2 categories.
![alt text](image.png)

In [ ]:
#maybe not necessary, because all Values are integers (no str for categories)
# Identify the categorical (cat_cols) and numerical features (num_cols)
#num_cols = ['BMI', 'Age']
#cate_cols = data.columns.drop(['BMI'])

# Data preprocessing and visualisation

### Dublicates. Missing Values and Outliers

In [53]:
#check data's integrity
data.duplicated()
int(data.duplicated().sum())


23899

This are a lot of duplicates but I assume that this happens verry quick, even if it is really a different patient, with such a huge amount of categorical 2 group variables. Because of this it would be bad to drop dublicates and keep respectively the first of them.

In [54]:
#check for missing values
data.isna().sum(axis=1)
data.isna().any(axis=1).any()

np.False_

<- no missing values

### Outliers!!! -> needs to be included

In [ ]:
#does Chol Check ever has a value 0? Then the patient would not been tested and HighChol would be NAN rather than the value there.
sum(data['CholCheck']==0) #but: 0 = no cholesterol check in 5 years 1 = yes cholesterol check in 5 years
#maybe they checked this value earlier than 5 years in past

9470

In [56]:
"""Another important command to be aware of is [`groupby()`]
(https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.groupby.html), 
which allows you to apply a function or perform a computation for a particular view of a `DataFrame`. 
For example, if we want to compute the total number of children orphaned due to AIDS for every country, 
we can use the following command:

data_cleaned.groupby("Country")["Data.AIDS-Related Deaths.AIDS Orphans"].sum()"""

'Another important command to be aware of is [`groupby()`]\n(https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.groupby.html), \nwhich allows you to apply a function or perform a computation for a particular view of a `DataFrame`. \nFor example, if we want to compute the total number of children orphaned due to AIDS for every country, \nwe can use the following command:\n\ndata_cleaned.groupby("Country")["Data.AIDS-Related Deaths.AIDS Orphans"].sum()'

### Data Visualisation

In [ ]:
""" from Tutorial 2
sns.displot(
    x="Data.AIDS-Related Deaths.All Ages",
    data=data_cleaned
)
plt.show() """